In [ ]:
!pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

path_chromedriver='C:/Projetos/env/chromedriver/chromedriver.exe'
s = Service(path_chromedriver)

driver = webdriver.Chrome(service=s)